In [26]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm
from find_files import corsika_data_files
from process_data import pdg_mass_maps
from corsika2dict import Corsika2Dict, join_dicts
from save2hdf5 import save_dict_to_hdf5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
data_files = corsika_data_files("34_simple_run")

data_files = data_files[0:1]

In [32]:
pdgs_to_get = [-12, 12, -13, 13, -14, 14]
theta = 30

In [37]:
data_files[0].parents[1]

PosixPath('/dicos_ui_home/antonpr/work/corsika_runs/34_simple_run')

In [33]:
maps = pdg_mass_maps()
results = []
for file in tqdm(data_files, total = len(data_files)):
    corsika_dict = Corsika2Dict(file, theta, maps)
    results.append(corsika_dict.distr_dict(pdgs_to_get))

100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


In [34]:
save_dict_to_hdf5(join_dicts(results), data_files[0].parents[1]/"test.hdf5")

In [ ]:
corsika_results_dict = {}

        for pdg in pdgs_to_get:
            corsika_results_dict[pdg] = {}
            for obslev_id in obs_levels_range:
                corsika_results_dict[pdg][obslev_id] = {'theta [rad]': [], 
                                                        'energy [GeV]': []}
            
                filtered_ids = np.where((self.pdgs == pdg) & (self.obs_levels == obslev_id))[0]
                corsika_results_dict[pdg][obslev_id]['theta [rad]'] = self.angles[filtered_ids]
                corsika_results_dict[pdg][obslev_id]['energy [GeV]'] = self.kin_energy[filtered_ids]  
        
        corsika_results_dict['num_primaries'] = np.int64(self.num_primaries)

In [25]:

corsika_results_dict = {}
res0 = result[0]
pdgs_to_get = list(res0.keys())[:-1]
first_key = pdgs_to_get[0]
obs_level_num = len(res0[pdgs_to_get[0]])

for pdg in pdgs_to_get:
    corsika_results_dict[pdg] = {}
    for obslev_id in range(obs_level_num):
        corsika_results_dict[pdg][obslev_id] = {'theta [rad]': [], 
                                                        'energy [GeV]': []}
        angles = []
        energies = []
        for res in result:
            angles.append(res[pdg][obslev_id]['theta [rad]'])
            energies.append(res[pdg][obslev_id]['energy [GeV]'])

        corsika_results_dict[pdg][obslev_id]['theta [rad]'] = np.concatenate(angles)
        corsika_results_dict[pdg][obslev_id]['energy [GeV]'] = np.concatenate(energies) 

num_primaries = 0
for res in result:
    num_primaries += res['num_primaries']

corsika_results_dict['num_primaries'] = np.int64(num_primaries)    
    
    

# print(corsika_results_dict['num_primaries'])
# print(len(corsika_results_dict[-12][0]['theta [rad]']))
# print(len(corsika_results_dict[-12][0]['energy [GeV]']))

[-12, 12, -13, 13, -14, 14] 4


In [134]:
save_dict_to_hdf5(res, "15_test.hdf5")